# 使用 Nsight Compute 优化在线 5G 机器学习

## 07 平衡内存和计算

在之前优化将基础向量移至共享内存以便更好地重复使用之后，我们仍然面临“屏障”扭曲停滞的问题。

在之前的版本中，“从全局内存到共享内存的所有加载都是在循环开始时由单个扭曲完成的”。在此期间，“其余的扭曲处于空闲（停滞）状态”，浪费了在“屏障”处等待的宝贵周期。这可以通过两种方式进行改进：我们可以读取较小的批次并将其与计算交替进行；我们可以“通过并行发出更多访问来执行更高效的读取”。对于后者，我们使用其他样本的扭曲。通过这种方式，我们的目标是平衡内存和计算操作。此实现的另一个结果是，我们不再将输入的长度限制为最大向量长度，因为算法现在与向量大小无关。

您知道该怎么做：将 [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h) 中的 `APSM_DETECT_VERSION` 标志设置为 `apsm_version::APSM_DETECT_BALANCED`，然后使用以下命令重新编译代码并收集报告：

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!make -j
!ncu -k kernel_apsm_detect --set full --import-source yes -f -o /dli/task/ncu/report_balanced \
    bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

最后一次，在 Nsight Compute UI 中打开新的报告文件 `/root/Desktop/reports/ncu/report_balanced.ncu-rep` 。

同样，我们的优化对内核运行时产生了非常积极的影响，将其减少了 60%。此外，计算和内存利用率现在非常平衡，都在 67% 左右。此时，该工具认为内核不再受延迟限制，

<img src="images/ncu_report04_01.png">

在 `Scheduler Statistics` 部分也可以看到相同的延迟减少，其中 `Issued Warp Per Scheduler` 提高了 76%，现在约为 37%。对该指标的改进通常会对内核性能产生非常积极的影响。

<img src="images/ncu_report04_02.png">

最后，`Warp States` 图表突出显示“Barrier 停顿已完全可以忽略不计”（而“MIO Throttle”停顿有所增加）。

当然，可以进行进一步的优化。MIO Throttle 表示我们仍然过于积极地访问内存。经过仔细检查，我们可以看到存在大量共享内存库冲突（查看内存工作负载分析部分：向量大小会导致冲突。为了缓解这种情况，我们可以添加填充，这应该可以减少库冲突的数量。

事实上，在现实世界中，我们的优化目标是目标硬件上的运行时间少于 1 毫秒。但是，我们认为内核现在足够快，可以满足本次训练的目的。

您现在可以转到此 Nsight Compute 教程的 [总结](08_summary.ipynb) 。